In [1]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow.data import Dataset
# from vae import VAE

In [2]:
import matplotlib.pyplot as plt
from skimage import color
from skimage import io
from skimage.transform import resize
import os
from glob import glob
import os.path as path

In [3]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected as fc
class VariantionalAutoencoder(object):

    def __init__(self, learning_rate=1e-4, batch_size=64, n_z=16):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_z = n_z

        tf.reset_default_graph()
        self.build()

        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())

    # Build the netowrk and the loss functions
    def build(self):
        self.x = tf.placeholder(
            name='x', dtype=tf.float32, shape=[None, input_dim])

        # Encode
        # x -> z_mean, z_sigma -> z
        f1 = fc(self.x, 256, scope='enc_fc1', activation_fn=tf.nn.elu)
        f2 = fc(f1, 128, scope='enc_fc2', activation_fn=tf.nn.elu)
        f3 = fc(f2, 64, scope='enc_fc3', activation_fn=tf.nn.elu)
        self.z_mu = fc(f3, self.n_z, scope='enc_fc4_mu', 
                       activation_fn=None)
        self.z_log_sigma_sq = fc(f3, self.n_z, scope='enc_fc4_sigma', 
                                 activation_fn=None)
        eps = tf.random_normal(
            shape=tf.shape(self.z_log_sigma_sq),
            mean=0, stddev=1, dtype=tf.float32)
        self.z = self.z_mu + tf.sqrt(tf.exp(self.z_log_sigma_sq)) * eps

        # Decode
        # z -> x_hat
        g1 = fc(self.z, 64, scope='dec_fc1', activation_fn=tf.nn.elu)
        g2 = fc(g1, 128, scope='dec_fc2', activation_fn=tf.nn.elu)
        g3 = fc(g2, 256, scope='dec_fc3', activation_fn=tf.nn.elu)
        self.x_hat = fc(g3, input_dim, scope='dec_fc4', 
                        activation_fn=tf.sigmoid)

        # Loss
        # Reconstruction loss
        # Minimize the cross-entropy loss
        # H(x, x_hat) = -\Sigma x*log(x_hat) + (1-x)*log(1-x_hat)
        epsilon = 1e-10
        recon_loss = -tf.reduce_sum(
            self.x * tf.log(epsilon+self.x_hat) + 
            (1-self.x) * tf.log(epsilon+1-self.x_hat), 
            axis=1
        )
        self.recon_loss = tf.reduce_mean(recon_loss)

        # Latent loss
        # KL divergence: measure the difference between two distributions
        # Here we measure the divergence between 
        # the latent distribution and N(0, 1)
        latent_loss = -0.5 * tf.reduce_sum(
            1 + self.z_log_sigma_sq - tf.square(self.z_mu) - 
            tf.exp(self.z_log_sigma_sq), axis=1)
        self.latent_loss = tf.reduce_mean(latent_loss)

        self.total_loss = self.recon_loss + self.latent_loss
        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.total_loss)
        
        self.losses = {
            'recon_loss': self.recon_loss,
            'latent_loss': self.latent_loss,
            'total_loss': self.total_loss,
        }        
        return

    # Execute the forward and the backward pass
    def run_single_step(self, x):
        _, losses = self.sess.run(
            [self.train_op, self.losses],
            feed_dict={self.x: x}
        )
        return losses

    # x -> x_hat
    def reconstructor(self, x):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x})
        return x_hat

    # z -> x
    def generator(self, z):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z})
        return x_hat
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z


In [20]:
"""Define Dataset and global parameters"""
data_root=path.expanduser('/home/aitrading/Desktop/GLTransform/output/npy/')
np_files=glob(path.join(data_root,'*.npy'))  # list
mode = 'fully_connected'
num_sample = 1400
input_dim = 512*512

def generate(batch_size):
    n=len(np_files)
    i=0
    while True:
        temp = []
        
        for j in range(batch_size):
            i+=1
            if i>=n:
                return
            temp.append(np.load(np_files[i]).reshape(-1))
        
        yield np.array(temp)
        

In [5]:
# if mode == 'fully_connected':
#     data_set = Dataset.from_generator(generate, (tf.int32, tf.int32))
# elif mode == 'CNN':
#     data_set = Dataset.from_generator(generate, (tf.int32, tf.int32), output_shapes = tf.TensorShape([512,512]))

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [57]:
# def trainer(model_object, num_sample, input_dim, learning_rate=1e-4, batch_size=64, num_epoch=5, n_z=16, log_step=5, mode=mode):
#     model = model_object(mode= mode, input_dim=input_dim, learning_rate=learning_rate, batch_size=batch_size, no_z=n_z)

#     for epoch in range(num_epoch):
#         start_time = time.time()
#         for i in range(num_sample // batch_size):
#             # Get a batch

#             batch = next(generate())
#             print(batch.shape)
#             # Execute the forward and backward pass 
#             # Report computed losses
#             losses = model.run_single_step(batch)
#         end_time = time.time()
        
#         if epoch % log_step == 0:
#             log_str = '[Epoch {}] '.format(epoch)
#             for k, v in losses.items():
#                 log_str += '{}: {:.3f}  '.format(k, v)
#             log_str += '({:.3f} sec/epoch)'.format(end_time - start_time)
#             print(log_str)
            
#     print('Done!')
#     return model

In [24]:
# data_set = data_set.batch(32)
def trainer(model_object, learning_rate=1e-4, 
            batch_size=32, num_epoch=30, n_z=16, log_step=5):
    model = model_object(
        learning_rate=learning_rate, batch_size=batch_size, n_z=n_z)

    for epoch in range(num_epoch):
        start_time = time.time()
        for iter in range(num_sample // batch_size):
            # Get a batch
            batch = next(generate(batch_size))
#             batch = data_set.make_one_shot_iterator().get_next()
#             print(batch.shape)
    
            # Execute the forward and backward pass 
            # Report computed losses
            losses = model.run_single_step(batch)
        end_time = time.time()
        
        if epoch % log_step == 0:
            log_str = '[Epoch {}] '.format(epoch)
            for k, v in losses.items():
                log_str += '{}: {:.3f}  '.format(k, v)
            log_str += '({:.3f} sec/epoch)'.format(end_time - start_time)
            print(log_str)
            
    print('Done!')
    return model


# Train a model
model = trainer(VariantionalAutoencoder)

[Epoch 0] recon_loss: nan  latent_loss: nan  total_loss: nan  (2.440 sec/epoch)


KeyboardInterrupt: 

In [48]:
# model = trainer(VAE,num_sample,input_dim)

num_sample // batch_size 21
[Epoch 0] recon_loss: -inf  latent_loss: nan  total_loss: nan  (0.587 sec/epoch)
num_sample // batch_size 21
num_sample // batch_size 21
num_sample // batch_size 21
num_sample // batch_size 21
Done!


In [ ]:
def test_reconstruction(model, mnist, h=512, w=512, batch_size=16):
    # Test the trained model: reconstruction
    batch = data_set.take(1)
    x_reconstructed = model.reconstructor(batch[0])

    n = np.sqrt(batch_size).astype(np.int32)
    I_reconstructed = np.empty((h*n, 2*w*n))
    for i in range(n):
        for j in range(n):
            x = np.concatenate(
                (x_reconstructed[i*n+j, :].reshape(h, w), 
                 batch[0][i*n+j, :].reshape(h, w)),
                axis=1
            )
            I_reconstructed[i*h:(i+1)*h, j*2*w:(j+1)*2*w] = x

    plt.figure(figsize=(10, 20))
    plt.imshow(I_reconstructed, cmap='gray')